# AI1
## This is the project No.1 and it use to find the optimal path in a game.



### Summary:

This is a game which a character named <b>Gandalf</b> should go to destination(<b>Gondor</b>) from the start state. In the way, it may see some characters named <b>Orak</b> with military rank n that they monitor cells with a Manhattan distance of n. Before Gandalf reachs Gondor, it should find <b>assistants</b> and put them in their <b>destinations</b> and also they should take care the oraks restrictions. 

### Description:

Gray Gandalf confronts Sauron the Dark Lord with the goal of leading and guiding the allies of the mission ring. To do this, he must pass the allies of the ring through the Oraks, which are Sauron's army, and bring each one to a specific position, and at the end he goes to Gondor. Gandalf has checkered patterns that include the following:
<ul>
    <li>
Gandalf started state
    </li>
    <li>    
Position of each of the ring assistants
    </li>        
    <li>        
Position of orcs
    </li>        
    <li>        
The place where each ring friend should be placed
    </li>       
    <li>       
Final position (Gondor)
</ul>

Each Orak has a military rank of n, according to which it monitors houses with a Manhattan distance of n that Gandalf and his companions cannot pass. Because Gandalf is supposed to pass through the oraks at night and they are asleep, it can pass through the houses they monitor as long as they do not wake them up. After passing n houses in the area under the supervision of the Oraks, they wake up. That is, by moving the 1 + n, we have to go outside the range. After leaving the area under the supervision of the orcs, you can return to that area without considering the previous moves. In each move, Gandalf can move to one of the four houses adjacent to the current house, ie up, down, left and right, provided that the destination house is outside
There is no map area and there is no Oraks in it. Gandalf picks up the house where the ring companions are and picks him up, and after reaching the ring's location, he settles him there without spending time. Gandalf can only have one of the ring's companions at a time. The position of the assistants and the place where they should be located are never within the control of the Oraks and the two areas under the supervision of the Oraks are not shared.

In [1]:
from time import time
import numpy as np
import pandas as pd
from statistics import mean

### Defining Constants
<ul>
    <li>
        Determine each character with a number to use in the table
    </li>
    <li>
        Determine four directions
    </li>
    <li>
        Determine two mode for algorithms we use for helper or helper's destination
    </li>    
    <li>
        Determine three mode BFS, IDS and A*
    </li>     
</ul>

In [2]:
GANDALF = -6
HELPER = -12
ORAK = -8
GOAL = -9
ORAK_REGION = -10
HELPER_DES = -11

FIND_HELPER_MODE = -1
FIND_DES_MODE = -2

DIRECTION_COUNT = 4

LEFT = 0
RIGHT = 1
UP = 2
DOWN = 3

LEFT_DIR = (0, -1)
RIGHT_DIR = (0, 1)
UP_DIR = (-1, 0)
DOWN_DIR = (1, 0)

REACH_HELPER_MODE = 1
REACH_DES_MODE = 2

BFS_MODE = 0
IDS_MODE = 1
A_STAR_MODE = 2

EXECUTE_COUNT = 3

ALPHA1 = 2
ALPHA2 = 10

## Import Data
### import data by read_data function
<ul>
    <li>
In the first row, it is stated n and m, respectively, that n is the number of rows in the page and m is the number of columns.
    </li>
    <li>
The following two lines show the coordinates of the start and end points, respectively. First the row number from the top then the column number from the left
    </li>
    <li>        
In the next line k and l it is stated that k represents the number of orcs and l the number of loop helpers.
K In the next k line, x, y, and c, it is stated that x is the row number, y is the column number, and c is the military rank of the orc, or the Manhattan distance under the supervision of an orc.
    </li>
    <li>        
In the next line, the current location of the ring assistants is listed in order.
    </li>
    <li>        
In the next line l, in the same order as before, the place where the circle helpers should be located is given. Note that each ring friend must be in a specific location. That is, the friend of the i ring should be in the i place.
    </li>
</ul>

In [3]:
def read_data(filename):
    row_count = 0
    col_count = 0
    initial_state = ()
    goal_state = ()
    orak_count = 0
    helper_count = 0
    oraks = []
    helpers = []
    helpers_des = []
    with open(filename) as f:
        row_count, col_count = [int(x) for x in next(f).split(' ')]
    
        start_x, start_y = [int(x) for x in next(f).split(' ')]
        initial_state = (start_x, start_y)
        
        finish_x, finish_y = [int(x) for x in next(f).split(' ')]
        goal_state = (finish_x, finish_y)
        
        orak_count, helper_count = [int(x) for x in next(f).split(' ')]
    
        for item in range(orak_count):
            orak_x, orak_y, rank = [int(x) for x in next(f).split(' ')] 
            oraks.append((orak_x, orak_y, rank))
            
        for item in range(helper_count):
            helper_x, helper_y = [int(x) for x in next(f).split(' ')] 
            helpers.append((helper_x, helper_y))
            
        for item in range(helper_count):
            helper_des_x, helper_des_y = [int(x) for x in next(f).split(' ')] 
            helpers_des.append((helper_des_x, helper_des_y))      
            
    return row_count, col_count, initial_state, goal_state, orak_count, helper_count, oraks, helpers, helpers_des

### LIFO & FIFO
make two classes named LIFO and FIFO to use in order in IDS and BFS

In [4]:
class FIFO_queue:
    def __init__(self):
        self.dataset = []

    def enqueue(self, data):
        self.dataset.append(data)

    def dequeue(self):
        if self.is_empty() == True:
            return
        
        data = self.dataset[0]
        self.dataset.pop(0)

        return data

    def initialize(self, data):
        self.enqueue(data)

    def is_empty(self):
        if len(self.dataset) == 0:
            return True
        
    def clear(self):
        self.dataset.clear()

class LIFO_queue:
    def __init__(self):
        self.dataset = []

    def enqueue(self, data):
        self.dataset.append(data)

    def dequeue(self):
        if self.is_empty() == True:
            return
        data = self.dataset[len(self.dataset)-1]

        self.dataset.pop(len(self.dataset)-1)

        return data

    def initialize(self, data):
        self.enqueue(data)

    def is_empty(self):
        if len(self.dataset) == 0:
            return True
        return False

# BFS Functions
-------------------------------------------------------

### Calculate path cost
Define calculate_path function to write optimal path on output

In [5]:
def calculate_path(child, path, path_cost, parents, direction_table, initial_state):
    new_path = ''
    while child != initial_state:
        if direction_table[child[0]][child[1]][0] == LEFT:
            new_path += 'L'
        elif direction_table[child[0]][child[1]][0] == RIGHT:
            new_path += 'R'
        elif direction_table[child[0]][child[1]][0] == UP:
            new_path += 'U'
        elif direction_table[child[0]][child[1]][0] == DOWN:
            new_path += 'D'
        path_cost += 1    
            
        child = parents[child]    
    path += new_path[::-1]
    return path, path_cost

## Define BFS 
Implement BFS and its action that should to be done for four directions and check four children's states.
<ul>
    <li>
        initial_node is the first cell
    </li>
    <li>
        visited_states is a matrix to determine seen cells like explored
    </li>
    <li>
        frontier is a FIFO queue to check remain cells
    </li>    
    <li>
        direction_table is a matrix that has two element; first for determine direction that this cell reached and second is used for Orak's rank and restriction
    </li>     
    <li>
        has_reached_goal is a boolean variable to show that Gandalf has arrived in goal state or not
    </li>      
</ul>

In [6]:
def action(visited_states, table, node, parents, frontier,
           goal_state, direction_table, has_reached_goal,
           initial_state, path, path_cost, mode, oraks):
    curr_dir = set()
    
    for direction in range(DIRECTION_COUNT):
        if direction == LEFT:
            curr_dir = LEFT_DIR
        elif direction == RIGHT:
            curr_dir = RIGHT_DIR
        elif direction == UP:
            curr_dir = UP_DIR
        elif direction == DOWN:
            curr_dir = DOWN_DIR  
            
        # Is in valid state    
        if not is_in_table_region(node[0], node[1], curr_dir[0], curr_dir[1], row_count, col_count):
            continue
        if visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]]: 
            continue
        if table[node[0] + curr_dir[0]][node[1] + curr_dir[1]] == ORAK:
            visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
            continue
        if isinstance(table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], int):
            if table[node[0] + curr_dir[0]][node[1] + curr_dir[1]] >= 0:
                if direction_table[node[0]][node[1]][1] == None:
                    direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], oraks[table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]][2] - 1)
                    
                elif direction_table[node[0]][node[1]][1][0] == table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]:
                    if direction_table[node[0]][node[1]][1][1] == 0:
                        visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
                        continue
                    else:
                        direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], direction_table[node[0]][node[1]][1][1] - 1)
                else:
                    direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], oraks[table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]][2] - 1)
                                        
        
        # Determine LRUD
        child = (node[0] + curr_dir[0], node[1] + curr_dir[1])
        direction_table[child[0]][child[1]][0] = direction
        visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
        parents[child] = node
        
        if mode == REACH_HELPER_MODE:
            if table[child[0]][child[1]] == goal_state:
                has_reached_goal = True
                table[child[0]][child[1]] = '_'
                path, path_cost = calculate_path(child, path, path_cost,
                                                 parents, direction_table, initial_state)
                initial_state = child
                return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost
        else:
            if child == goal_state:
                has_reached_goal = True
                path, path_cost = calculate_path(child, path, path_cost,
                                                 parents, direction_table, initial_state)
                initial_state = child
                return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost
    
        frontier.enqueue(child)
     
    return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost       
                
def BFS(table, row_count, col_count, initial_state,
        goal_state, helpers, path, path_cost, mode, oraks, seen_states_count):
    
    # Elements to use in BFS algorithm
    node = initial_state
    visited_states = [[ False for i in  range(col_count)] for j in range(row_count)]
    parents = {}
    explored = set()
    frontier = FIFO_queue()
    frontier.enqueue(initial_state)
    
    if table[node[0]][node[1]] == goal_state:
        table[node[0]][node[1]] == '_'
        return table, (node[0], node[1]), path, path_cost
        
    visited_states[node[0]][node[1]] = True
    
    # First element for direction and second element for oraks' restrictions
    direction_table = [[ [0, None] for i in  range(col_count)] for j in range(row_count)]

    has_reached_goal = False
    while not has_reached_goal:
        seen_states_count += 1
        node = frontier.dequeue()
        explored.add(node)
        visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost = action(
            visited_states, table, node, parents, frontier, 
            goal_state, direction_table, has_reached_goal, initial_state, path, path_cost, mode, oraks)
        if(has_reached_goal):
            return table, initial_state, path, path_cost, seen_states_count

### BFS
Initialize initial and goal state and do BFS on helpers, their destination and Gabdalf's destination

In [7]:
def run_with_BFS(table):
    initial_state = gandalf_start_state
    goal_state = HELPER
    
    seen_states_count = 0
    path = ''
    path_cost = 0
    for helper_num in range(helper_count):
        # Find a helper
        table, initial_state, path, path_cost, seen_states_count = BFS(table, row_count, col_count, initial_state, goal_state, helpers, path, path_cost, REACH_HELPER_MODE, oraks, seen_states_count)
    
        # reach founded helper's destination
        index = helpers.index(initial_state)
        goal_state = helpers_des[index]
        table, initial_state, path, path_cost, seen_states_count = BFS(table, row_count, col_count, initial_state, goal_state, helpers, path, path_cost, REACH_DES_MODE, oraks, seen_states_count)
    
        goal_state = HELPER
        if helper_num == helper_count - 1:
            goal_state = gandalf_goal_state

    if helper_count == 0:
        goal_state = gandalf_goal_state

    table, initial_state, path, path_cost, seen_states_count = BFS(table, row_count, col_count, initial_state, goal_state, helpers, path, path_cost, REACH_DES_MODE, oraks, seen_states_count)  
    
    return path, path_cost, seen_states_count

# IDS Functions
-------------------------------------------------------

### DFS
Define DFS to use in IDS for four direction that Gandalf can go and add to frontier

In [8]:
def DFS(visited_states, table, node, parents, frontier, 
            goal_state, direction_table, has_reached_goal, initial_state, path, path_cost, mode, oraks, parents_length):
    
    if parents_length[node] <= 0:
        return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost, parents_length
    
    curr_dir = set()
    
    for direction in range(DIRECTION_COUNT):
        
        if direction == LEFT:
            curr_dir = LEFT_DIR
        elif direction == RIGHT:
            curr_dir = RIGHT_DIR
        elif direction == UP:
            curr_dir = UP_DIR
        elif direction == DOWN:
            curr_dir = DOWN_DIR  
            
        # Is in valid state    
        if not is_in_table_region(node[0], node[1], curr_dir[0], curr_dir[1], row_count, col_count):
            continue
        if table[node[0] + curr_dir[0]][node[1] + curr_dir[1]] == ORAK:
            visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
            continue
        if visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]]:
            if parents_length[node] - 1 < parents_length[(node[0] + curr_dir[0], node[1] + curr_dir[1])]:
                continue

        if isinstance(table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], int):
            if table[node[0] + curr_dir[0]][node[1] + curr_dir[1]] >= 0:
                if direction_table[node[0]][node[1]][1] == None:
                    direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], oraks[table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]][2] - 1)
                    
                elif direction_table[node[0]][node[1]][1][0] == table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]:
                    if direction_table[node[0]][node[1]][1][1] == 0:
                        visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
                        parents_length[(node[0] + curr_dir[0], node[1] + curr_dir[1])] = -1
                        continue
                    else:
                        direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], direction_table[node[0]][node[1]][1][1] - 1)
                else:
                    direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], oraks[table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]][2] - 1)
                                        
        
        # Determine LRUD
        child = (node[0] + curr_dir[0], node[1] + curr_dir[1])
        direction_table[child[0]][child[1]][0] = direction
        visited_states[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
        parents[child] = node
        
        if mode == REACH_HELPER_MODE:
            if table[child[0]][child[1]] == goal_state:
                has_reached_goal = True
                table[child[0]][child[1]] = '_'
                path, path_cost = calculate_path(child, path, path_cost,
                                                 parents, direction_table, initial_state)
                initial_state = child
                return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost, parents_length
        else:
            if child == goal_state:
                has_reached_goal = True
                path, path_cost = calculate_path(child, path, path_cost,
                                                 parents, direction_table, initial_state)
                initial_state = child
                return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost, parents_length
    
        frontier.enqueue(child)
        parents_length[child] = parents_length[node] - 1
     
    return visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost, parents_length

## Define IDS
second algorithm we use to calculate optimal path
Implement BFS and its action that should to be done for four directions and check four children's states.
<ul>
    <li>
        initial_node is the first cell
    </li>
    <li>
        visited_states is a matrix to determine seen cells like explored
    </li>
    <li>
        frontier is a LIFO queue to check remain cells
    </li>    
    <li>
        direction_table is a matrix that has two element; first for determine direction that this cell reached and second is used for Orak's rank and restriction
    </li>     
    <li>
        has_reached_goal is a boolean variable to show that Gandalf has arrived in goal state or not
    </li>   
    <li>
        parents_length is a dic to check length restriction used to check that Gandalf can go more depth or not
    </li>     
</ul>

In [9]:
def IDS(table, row_count, col_count, initial_state,
        goal_state, helpers, path, path_cost, mode, oraks, seen_states_count):
    length = 1
    has_reached_goal = False
    const_initial_state = initial_state
    while not has_reached_goal:
        node = initial_state
        visited_states = [[ False for i in  range(col_count)] for j in range(row_count)]
        visited_states[node[0]][node[1]] = True
        explored = set()
        parents = {}
        frontier = LIFO_queue()
        frontier.enqueue(initial_state)
        direction_table = [[ [0, None] for i in  range(col_count)] for j in range(row_count)]
        parents_length = {}
        parents_length[node] = length
        

        while not frontier.is_empty():
            seen_states_count += 1
            node = frontier.dequeue()
            explored.add(node)
            visited_states, direction_table, parents, frontier, has_reached_goal, table, initial_state, path, path_cost, parents_length = DFS(visited_states, table, node, parents, frontier, goal_state,
                                                        direction_table, has_reached_goal, initial_state, path, path_cost, mode, oraks, parents_length)
        
            if has_reached_goal:
                return table, initial_state, path, path_cost, seen_states_count
            
        initial_state = const_initial_state
        length += 1

## IDS
Initialize initial and goal state and do IDS on helpers, their destination and Gabdalf's destination

In [10]:
def run_with_IDS(table):
    initial_state = gandalf_start_state
    goal_state = HELPER
    seen_states_count = 0
    path = ''
    path_cost = 0
    for helper_num in range(helper_count):
        # Find a helper
        table, initial_state, path, path_cost, seen_states_count = IDS(table, row_count, col_count, initial_state,
                                                    goal_state, helpers, path, path_cost, REACH_HELPER_MODE, oraks, seen_states_count)
    
        # reach founded helper's destination
        index = helpers.index(initial_state)
        goal_state = helpers_des[index]
        table, initial_state, path, path_cost, seen_states_count = IDS(table, row_count, col_count, initial_state,
                                                goal_state, helpers, path, path_cost, REACH_DES_MODE, oraks, seen_states_count)
    
        goal_state = HELPER
        if helper_num == helper_count - 1:
            goal_state = gandalf_goal_state

    if helper_count == 0:
        initial_state = gandalf_start_state
        goal_state = gandalf_goal_state

    table, initial_state, path, path_cost, seen_states_count = IDS(table, row_count, col_count, initial_state,
                                           goal_state, helpers, path, path_cost, REACH_DES_MODE, oraks, seen_states_count)  
    return path, path_cost, seen_states_count

# A* Functions
-------------------------------

## Calculate distance
This is used to calculate the taxicab geometry between two cells

In [11]:
def calculate_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

## Search for the best f
Find the least f between open_list items

In [12]:
def get_least_f_state(heuristic_pars, open_list):
    res = open_list[0]
    for item in range(1, len(open_list)):
        if heuristic_pars[open_list[item]][0] < heuristic_pars[res][0]:
            res = open_list[item]
    return res

## Define A*
Define A* and a function called add_curr_state_children to add the children of current_node that poped from open_list
<ul>
    <li>
        openlist is cells' list that should be checked
    </li>
    <li>
        visited_states is a matrix to determine closed cells like closedlist
    </li>
    <li>
        initial_node is the first cell
    </li>    
    <li>
        direction_table is a matrix that has two element; first for determine direction that this cell reached and second is used for Orak's rank and restriction
    </li>     
    <li>
        has_reached_goal is a boolean variable to show that Gandalf has arrived in goal state or not
    </li>   
    <li>
        heuristic_pars is a variable to save f, g and h. h is the least distance to goal state, g is the path cost that have traveled and f is equal to g plus h.
    </li>     
</ul>

In [13]:
def add_curr_state_children(closed_list, table, node, parents, open_list,
                            direction_table, initial_state, path, path_cost, oraks, heuristic_pars, goal_state, alpha):
    
    curr_dir = set()
    
    for direction in range(DIRECTION_COUNT):
        if direction == LEFT:
            curr_dir = LEFT_DIR
        elif direction == RIGHT:
            curr_dir = RIGHT_DIR
        elif direction == UP:
            curr_dir = UP_DIR
        elif direction == DOWN:
            curr_dir = DOWN_DIR  
            
        # Is in valid state    
        if not is_in_table_region(node[0], node[1], curr_dir[0], curr_dir[1], row_count, col_count):
            continue
        if closed_list[node[0] + curr_dir[0]][node[1] + curr_dir[1]]: 
            continue
        if table[node[0] + curr_dir[0]][node[1] + curr_dir[1]] == ORAK:
            closed_list[node[0] + curr_dir[0]][node[1] + curr_dir[1]] = True
            continue
        if isinstance(table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], int):
            if table[node[0] + curr_dir[0]][node[1] + curr_dir[1]] >= 0:
                if direction_table[node[0]][node[1]][1] == None:
                    direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], oraks[table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]][2] - 1)
                    
                elif direction_table[node[0]][node[1]][1][0] == table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]:
                    if direction_table[node[0]][node[1]][1][1] == 0:
                        ###
                        continue
                    else:
                        direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], direction_table[node[0]][node[1]][1][1] - 1)
                else:
                    direction_table[node[0] + curr_dir[0]][node[1] + curr_dir[1]][1] = (table[node[0] + curr_dir[0]][node[1] + curr_dir[1]], oraks[table[node[0] + curr_dir[0]][node[1] + curr_dir[1]]][2] - 1)
                                        
        
        # Add child to open_list
        child = (node[0] + curr_dir[0], node[1] + curr_dir[1])
        g = heuristic_pars[node][1] + 1
        h = alpha * calculate_distance(child[0], child[1], goal_state[0], goal_state[1])
        f = g + h
        curr_child_heuristic = (f, g, h)
        
        if open_list.count(child) != 0:
            pre_child = open_list.pop(open_list.index(child))
            if curr_child_heuristic[1] <= heuristic_pars[pre_child][1]:
                direction_table[child[0]][child[1]][0] = direction
                parents[child] = node
                open_list.append(child)
                heuristic_pars[child] = curr_child_heuristic
            else:
                open_list.append(pre_child)
                
        else:
            open_list.append(child)
            direction_table[child[0]][child[1]][0] = direction
            parents[child] = node
            heuristic_pars[child] = curr_child_heuristic
                

    return closed_list, direction_table, parents, open_list, table, initial_state, path, path_cost, heuristic_pars


def A_star(table, row_count, col_count, initial_state, goal_state, helpers, path, path_cost, mode, oraks, alpha, remain_helpers, seen_states_count):
    open_list = []
    closed_list = [[ False for i in  range(col_count)] for j in range(row_count)]
    open_list.append(initial_state)
    
    # Find nearest helper as goal_state
    if mode == REACH_HELPER_MODE:
        nearest_helper = remain_helpers[0]
        for index in range(1, len(remain_helpers)):
            if calculate_distance(initial_state[0], initial_state[1], remain_helpers[index][0], remain_helpers[index][1]) < calculate_distance(initial_state[0], initial_state[1], nearest_helper[0], nearest_helper[1]):
                nearest_helper = remain_helpers[index]
        goal_state = nearest_helper
        remain_helpers.pop(remain_helpers.index(nearest_helper))

    direction_table = [[ [0, None] for i in  range(col_count)] for j in range(row_count)]
    parents = {}
    heuristic_pars = {}
    
    # define dic's second parameter as (f, g, h)
    heuristic_pars[initial_state] = (calculate_distance(initial_state[0], initial_state[1], goal_state[0], goal_state[1]) + 0, 0, calculate_distance(initial_state[0], initial_state[1], goal_state[0], goal_state[1]))
    
    while(len(open_list) != 0 ):
        current_state = get_least_f_state(heuristic_pars, open_list)
        open_list.remove(current_state)
        closed_list[current_state[0]][current_state[1]] = True
        seen_states_count += 1
        
        # Check cuurent node is  or not
        if current_state == goal_state:
            if mode == REACH_HELPER_MODE:
                table[current_state[0]][current_state[1]] = '_'
            path, path_cost = calculate_path(current_state, path, path_cost,
                                             parents, direction_table, initial_state)
            
            initial_state = current_state
            return table, initial_state, path, path_cost, remain_helpers, seen_states_count
        
        closed_list, direction_table, parents, open_list, table, initial_state, path, path_cost, heuristic_pars = add_curr_state_children(closed_list, table, current_state, parents, open_list, direction_table, initial_state, path, path_cost, oraks, heuristic_pars, goal_state, alpha)

## A*
Initialize initial and goal state and do IDS on helpers, their destination and Gabdalf's destination. Also here we define a variable named remain_helpers to do not check a helper twice in A* algorithm

In [14]:
def run_with_A_star(table, alpha = 1):
    initial_state = gandalf_start_state
    goal_state = HELPER
    remain_helpers = helpers.copy()
    seen_states_count = 0
    path = ''
    path_cost = 0
    for helper_num in range(helper_count):
        # Find a helper
        table, initial_state, path, path_cost, remain_helpers, seen_states_count = A_star(table, row_count, col_count, initial_state,
                                                    goal_state, helpers, path, path_cost, REACH_HELPER_MODE, oraks, alpha, remain_helpers, seen_states_count)
    
        # reach founded helper's destination
        index = helpers.index(initial_state)
        goal_state = helpers_des[index]
        table, initial_state, path, path_cost, remain_helpers, seen_states_count = A_star(table, row_count, col_count, initial_state,
                                                goal_state, helpers, path, path_cost, REACH_DES_MODE, oraks, alpha, remain_helpers, seen_states_count)
    
        goal_state = HELPER
        if helper_num == helper_count - 1:
            goal_state = gandalf_goal_state
            
    if helper_count == 0:
        initial_state = gandalf_start_state
        goal_state = gandalf_goal_state
        
    table, initial_state, path, path_cost, remain_helpers, seen_states_count = A_star(table, row_count, col_count, initial_state, goal_state, helpers, path, path_cost, REACH_DES_MODE, oraks, alpha, remain_helpers, seen_states_count)    
    
    return path, path_cost, seen_states_count

# Table Functions
-------------------------------------------------------

## Table restriction
Use to avoid exiting table

In [15]:
def is_in_table_region(curr_x, curr_y, dx, dy, row_count, col_count):
    if curr_x + dx < 0 or curr_x + dx >= row_count or curr_y + dy < 0 or curr_y + dy >= col_count:
        return False
    return True

### Make a table for current testcase

In [16]:
def define_table(row_count, col_count, orak_count, helper_count, oraks, helpers):
    table = [[ '_' for i in  range(col_count)] for j in range(row_count)]
    
    # Determine Oraks and their regions in the table
    for orak_num in range(orak_count):
        table[oraks[orak_num][0]][oraks[orak_num][1]] = ORAK
    
        curr_rank = oraks[orak_num][2]
        for i in range(-curr_rank - 1, curr_rank + 1):
            for j in range(-curr_rank - 1, curr_rank + 1):
                if (abs(i) + abs(j) <= curr_rank) and (i != 0 or j != 0):
                    if is_in_table_region(oraks[orak_num][0], oraks[orak_num][1], i, j, row_count, col_count):
                        table[oraks[orak_num][0] + i][oraks[orak_num][1] + j] = orak_num
                        
    # Determine helpers in the table     
    for helper_num in range(helper_count):
        table[helpers[helper_num][0]][helpers[helper_num][1]] = HELPER                        
    return table

# Testcases

In [17]:
def calculate_output_vars(mode, alpha=1):
    times = []
    if mode == BFS_MODE:
        for i in range(EXECUTE_COUNT):
            table = define_table(row_count, col_count, orak_count, helper_count, oraks, helpers)
            
            start = time()
            path, path_cost, seen_states_count = run_with_BFS(table)
            finish = time()
            times.append(finish - start)
        return path, path_cost, mean(times), seen_states_count
    elif mode == IDS_MODE:
        for i in range(EXECUTE_COUNT):
            table = define_table(row_count, col_count, orak_count, helper_count, oraks, helpers)
            
            start = time()
            path, path_cost, seen_states_count = run_with_IDS(table)
            finish = time()
            times.append(finish - start)
        return path, path_cost, mean(times), seen_states_count
    elif mode == A_STAR_MODE:
        for i in range(EXECUTE_COUNT):
            table = define_table(row_count, col_count, orak_count, helper_count, oraks, helpers)
            
            start = time()
            path, path_cost, seen_states_count = run_with_A_star(table, alpha)
            finish = time()
            times.append(finish - start)
        return path, path_cost, mean(times), seen_states_count

In [18]:
def run_testcase():
    res = []
    alpha1 = ALPHA1
    alpha2 = ALPHA2

    path, path_cost, time_average, seen_states_count = calculate_output_vars(BFS_MODE)
    BFS_res = [path, path_cost, time_average, seen_states_count]
    res.append(BFS_res)
  
    path, path_cost, time_average, seen_states_count = calculate_output_vars(IDS_MODE)
    IDS_res = [path, path_cost, time_average, seen_states_count]
    res.append(IDS_res)
    
    path, path_cost, time_average, seen_states_count = calculate_output_vars(A_STAR_MODE)
    A_star_res = [path, path_cost, time_average, seen_states_count]
    res.append(A_star_res)
    
    path, path_cost, time_average, seen_states_count = calculate_output_vars(A_STAR_MODE, alpha1)
    weighted_A_star_res = [path, path_cost, time_average, seen_states_count]
    res.append(weighted_A_star_res)
    
    path, path_cost, time_average, seen_states_count = calculate_output_vars(A_STAR_MODE, alpha2)
    weighted_A_star_res = [path, path_cost, time_average, seen_states_count]
    res.append(weighted_A_star_res)    

    df = pd.DataFrame(res, index=['BFS','IDS','A*',f'weighted A* alpha = {alpha1}', f'weighted A* alpha = {alpha2}'], columns=['path','path cost','time average','seen states count'])
    return df

## Testcase 1

In [19]:
row_count, col_count, gandalf_start_state, gandalf_goal_state, orak_count, helper_count, oraks, helpers, helpers_des = read_data('sample_testcases/test_00.txt')
res = run_testcase()
res

,path,path cost,time average,seen states count
BFS,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR,48,0.001332,163
IDS,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDURDURDURDURDRRRRRR,48,3.739439,623592
A*,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDURDURDURDURDRRRRRR,48,0.000000,63
weighted A* alpha = 2,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR,48,0.000000,59
weighted A* alpha = 10,RRRRRRRRRRRRRRRRRRRRRRRRRRRRRDRUDRUDRUDRUDRRRRRR,48,0.000000,59


## Testcase 2 

In [20]:
row_count, col_count, gandalf_start_state, gandalf_goal_state, orak_count, helper_count, oraks, helpers, helpers_des = read_data('sample_testcases/test_01.txt')
res = run_testcase()
res

,path,path cost,time average,seen states count
BFS,RRRRDDLLDDDDDLDRRRUUUURRULLDDDLDDRRRRRUUUUDDDD...,52,0.000000,260
IDS,DRRDRRDDDDDDLLLURUUUURRRRDLDLDDDLUURUURRRRDDDD...,52,0.021246,4061
A*,DRRDRRDDDDDDLLLURUUUURRRRDLDLDDDLUURUURRRRDDDD...,52,0.000997,134
weighted A* alpha = 2,RRRRDDLLDDDDDLDRRRUUURURULLDDDLDDRRRRRUUUULLLD...,54,0.000665,76
weighted A* alpha = 10,RRRRDDLLDLDDLDDRDRRRUUURURULLDDDLDDRRRRRUUUULL...,56,0.000665,73


## Testcase 3 

In [21]:
row_count, col_count, gandalf_start_state, gandalf_goal_state, orak_count, helper_count, oraks, helpers, helpers_des = read_data('sample_testcases/test_02.txt')
res = run_testcase()
res

,path,path cost,time average,seen states count
BFS,RRRRRRRLLLLDDDDDLLLRDDRUUURRRRDRDD,34,0.000332,108
IDS,RRRRRRRDDDDLLLLDLLLDDRUURURRRDRDDR,34,0.018295,4095
A*,RRRRRRRDDDDLLLLDLLLDDRUURURRRDRDDR,34,0.000666,87
weighted A* alpha = 2,RRRRRRRLLLLDDDDLDLLRDDRUURURRRDRDD,34,0.000332,57
weighted A* alpha = 10,RRRRRRRLLLLLDDRDDLDLLRDDRUURURRRDRDD,36,0.000337,54


## Testcase 4 

In [22]:
row_count, col_count, gandalf_start_state, gandalf_goal_state, orak_count, helper_count, oraks, helpers, helpers_des = read_data('sample_testcases/test_03.txt')
res = run_testcase()
res

,path,path cost,time average,seen states count
BFS,RRDDDDDDLLDDRDRRRRRRRRUUUUUUUUULLLLLLLDDDDDDDD...,66,0.001988,306
IDS,DDRDRDDDDDLLDRURUUUUUUUURRRRRRRLLDDDDDLLDDDDLL...,66,0.172909,39527
A*,DDRDRDDDDDLLDRURUUUUUUUURRRRRRRLLDDDDDLLDDDDLL...,66,0.003319,204
weighted A* alpha = 2,RRDDDDDLDLDDRDRRRRRRRRUUUUUUUUULLLLLLLDDDDDDDD...,66,0.000000,86
weighted A* alpha = 10,RRDDDDDLDLDDRDRRRRRRRRUUUUUUUUULLLLLLLLDDDRDDD...,68,0.000000,85


----------------------
# Concolusion and Some Explanation about Algorithms
In BFS we search for near cells and for each state, first cell has been appended in frontier poped and check its children to find the goal and if it doesn't find the goal, the children will be appended in frontier to check in next steps.<br/>
In IDS, we define a length to check that for each time, how many depth we can go and do DFS for every length we go to find the goal state. If the goal state doesn't be found, increase length and do DFS again for new length.<br/>
In A*, we define a heuristic_pars that is a variable to save f, g and h. h is the least distance to goal state, g is the path cost that have traveled and f is equal to g plus h. we use them to find the best way to go to the goal state. When we go to wach cell and then check their children, we don't check children's state with goal state; we just do that with the the current cell that is poped from openlist.<br/>

The implemented heuristic is the the sum of two cells' x distance with  two cells' y distance. It is consistent because for each cell we approach, we are getting closer to the goal state so the f value along a path increases or equals to previous cell and this is a proof that this heuristic is consistent.<br/>

BFS, IDS and A* find the best path cost for question and their answer are optimal(as seen in test cases' results) but they are different in seen states count; because although these algorithms are used to find optimal path, but their surface are different in states that go.<br/>

Here we see that IDS see too many states to find the optimal way and in result, it takes more time than other algorithms. In weighted A*, this algorithm prefer to reach goal by see less states and spend less time but the path may not be optimal.<br/>
In <b>time spending</b>, we can usually have this comparation:    A* &le; BFS &le; IDS